In [1]:
import pandas as pd
import scryfall
"""
fastest probably to manually download all card file, save JSON to this folder and replace the filename below
make sure to update the file below in order to have current price and card data
price data is considered stale after 24 hours
"""

#takes a json file downloaded from scryfall
#fname = "all-cards-20210224101318.json"
fname = "default-cards-20210318210305.json"
#takes a .dek xml file exported from MTGO
dekname = "Full_Trade_List.dek"

basiclands = ["Plains","Island","Swamp","Mountain","Forest"]

#todo
# 1. done (stripped done to which rows have at least either a mtgo_id or mtgo_foil_id)
# 2. done (selected which columns to keep)
# 3. done (load mtgo .dek file)
# 4. done (get prices for each item and then create a column and that dataframe with the price)
# 5. clean up and simplify the code & comment better
# 6. done (add interesting data print out in a final cell)
# 7. done(add price of tix to file)
# 8. fix prices for foils

In [2]:
#takes a while, loads in the scryfall file, drops some of the less important columns
scry_df = scryfall.read_scry(fname)
collection = scryfall.read_dek(dekname)


In [ ]:
#shrinks the scryfall file to just stuff that we want to work with
simpler_cols = pd.Series(['mtgo_id','mtgo_foil_id','prices'])
small_scry = scry_df[simpler_cols]#.dropna(thresh=2,subset=["mtgo_id","mtgo_foil_id"])

In [ ]:
#joining the two tables via the mtgo id value
collection = collection.set_index('mtgo_id').join(small_scry.set_index('mtgo_id'))
#when we joined the collection with the price guide, i set the index to be the mtgo ID, undoing that now
collection = collection.reset_index()

In [ ]:

#for each price dict in the price column, we are extracting the 'tix' value where there is one

tix = list()
for index,row in collection.iterrows():
    if type(row['prices']) == dict:
        tix.append(float(row['prices']['tix']))
    elif row['Name'] == "Event Ticket":
        tix.append(1)
    elif row['mtgo_id'] == 249:
        tix.append(.05) #this island isn't in the database maybe? skip it for now come back to checking on this
    else: #find foil price by going back and finding the row with the foil id and then getting the price from that line
        foil_price = small_scry[small_scry['mtgo_foil_id']==row['mtgo_id']]['prices'].values[0]['tix']
        tix.append(foil_price)
#add a new column that has the tix values for each line
collection['tix'] = pd.Series(tix)

In [ ]:
# Returns total value of collection in Tix (~USD)
collection['Quantity'] = collection['Quantity'].astype('int')

print( "Value in Tix: " +str((collection['tix'] * collection['Quantity']).sum() ))

#Average Value
print("avg value:" + str(collection['tix'].mean()))
print("mode value:" + str(collection['tix'].mode()[0]))

#Most valuable cards:
print("\nTop 5 most valuable cards")
print(collection[['tix','Name']].sort_values('tix',ascending=False).head(5))

#Number of cards worth 2 cents or less (not actually the legal PD cards)
print("\nNumber of penny cards: " + str(len(collection[collection['tix']<=.02])))

#counts of unique basic lands
print("\nNumber of unique basic lands: \n" + str(collection[collection['Name'].isin(basiclands)]['Name'].value_counts()))

#most common printing
print("\nMost common printing\n" + str(collection[collection['Quantity']==collection['Quantity'].max()]['Name']))

#most common card
print("\nMost common cards\n" + str(collection.groupby('Name')['Quantity'].sum().nlargest(15)))

In [ ]:
scry_df[scry_df['name']=='Black Lotus']

In [ ]:
mtgo_cards = scry_df[scry_df["mtgo_id"].notna()]
mtgo_foils = mtgo_cards[mtgo_cards["mtgo_foil_id"].notna()]

In [ ]:
mtgo_foils[mtgo_foils['mtgo_id']+1 != mtgo_foils['mtgo_foil_id']]

In [ ]:
scry_df.head()

In [ ]:
test['prices']

In [ ]:
small_scry[small_scry["mtgo_foil_id"]==collection[collection["mtgo_id"==249]]["mtgo_id"]]#["prices"][0]["tix"]

In [ ]:
collection[collection['prices'].isna()].head()

In [ ]:
small_scry[small_scry["mtgo_foil_id"]==249]

In [ ]:
collection["mtgo_id"].head()

In [ ]:
scry_df[scry_df["mtgo_id"]==34716]

In [ ]:
collection[collection["mtgo_id"]==34716]

In [ ]:
small_scry[small_scry["mtgo_id"]==34716]

In [5]:
collection[~collection["mtgo_id"].isin(scry_df["mtgo_id"]) & ~collection["mtgo_id"].isin(scry_df["mtgo_foil_id"])]

,mtgo_id,Quantity,Name
1453,34716,1,Swamp
1454,34718,1,Forest
1455,34724,1,Plains
1506,36121,3,Priest of Titania
1638,37603,3,Liliana's Specter
1798,39621,4,Everflowing Chalice
2027,43500,4,Curse of Thirst
2357,48008,1,Reliquary Tower
2609,51539,2,Elvish Mystic
2828,53839,1,Bile Blight
